In [540]:
import numpy as np
import scipy.stats
import pickle
import time
import statsmodels.api

startTime=time.time()
# Load the saved ARIMA model from the file
with open('mean_Latency_model.pkl', 'rb') as file:
    mean_loaded_model = pickle.load(file)
with open('std_Latency_model.pkl', 'rb') as file:
    std_loaded_model = pickle.load(file)



In [580]:
minute=1
fiveSecondFromMinute=minute*12
# Make predictions using the loaded model
prediction_time=int(time.time()-startTime)
print(prediction_time)
mean_pred = mean_loaded_model.predict(prediction_time,prediction_time+fiveSecondFromMinute, typ="levels")
std_dev_pred = std_loaded_model.predict(prediction_time, prediction_time+fiveSecondFromMinute, typ="levels")

# Define threshold and time interval
threshold = 8


# Calculate cumulative probabilities for each time step within the interval
cumulative_probabilities = []
for mean, std_dev in zip(mean_pred[:], std_dev_pred[:]):
    probability = scipy.stats.norm.cdf(threshold, loc=mean, scale=std_dev)
    cumulative_probabilities.append(probability)

print(cumulative_probabilities)
cumulative_probabilities=np.array(cumulative_probabilities)
cumulative_probabilities = cumulative_probabilities[~np.isnan(cumulative_probabilities)]

# Combine probabilities (e.g., take the maximum)
combined_probability =1- np.prod(cumulative_probabilities)
combined_probability_max=np.max(1-cumulative_probabilities)
print(f"Probability of exceeding 1 time '{threshold} second' of latency in {minute} minute: {combined_probability*100:.6f}%")
print(f"Max instant Probability of exceeding {threshold} second of latency in {minute} minute: {combined_probability_max*100:.6f}%")

387
[0.9475299164734304, 0.9475296176723457, 0.9475294089607861, 0.9475290466350142, 0.9475288925813089, 0.9475286618748227, 0.9476907974348394, 0.9475857486293706, 0.9475989367820085, 0.9476033155424766, 0.9475998068539907, 0.9476003325236744, 0.9475997550622596]
Probability of exceeding 1 time '8 second' of latency in 1 minute: 50.344518%
Max instant Probability of exceeding 8 second of latency in 1 minute: 5.247134%
